# Survey Exploratory Data Analysis Report

This notebook performs an automated EDA on the `umfrage.xlsx` file containing survey responses from dental practice staff. All outputs are generated automatically via GitHub Actions.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Set plot style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load and Inspect Survey Data

In [2]:
# Load the survey data
df = pd.read_excel('umfrage.xlsx')

print(f"Survey Dataset Shape: {df.shape[0]} responses and {df.shape[1]} columns")
print("\nFirst 2 responses (basic info only):")
display(df[['ID', 'Name', 'Startzeit', 'Fertigstellungszeit']].head(2))

Survey Dataset Shape: 26 responses and 32 columns

First 2 responses (basic info only):


,ID,Name,Startzeit,Fertigstellungszeit
0,1,NaN,2025-10-23 14:33:46,2025-10-23 14:47:53
1,2,NaN,2025-10-23 15:09:21,2025-10-23 15:16:48


## 2. Data Quality Overview

In [3]:
print("Data Types Overview:")
print(f"Total columns: {len(df.columns)}")
print(f"Text columns: {df.select_dtypes(include=['object']).shape[1]}")
print(f"Numeric columns: {df.select_dtypes(include=['int64', 'float64']).shape[1]}")
print(f"DateTime columns: {df.select_dtypes(include=['datetime64']).shape[1]}")

Data Types Overview:
Total columns: 32
Text columns: 27
Numeric columns: 3
DateTime columns: 2


In [4]:
# Check for missing values in key columns
key_columns = ['ID', 'Name', 'Startzeit', 'Fertigstellungszeit']
missing_key = df[key_columns].isnull().sum()
print("Missing values in key columns:")
for col, missing in missing_key.items():
    print(f"{col}: {missing} ({missing/len(df)*100:.1f}%)")

Missing values in key columns:
ID: 0 (0.0%)
Name: 26 (100.0%)
Startzeit: 0 (0.0%)
Fertigstellungszeit: 0 (0.0%)


In [5]:
# Overall completion rate
total_possible_answers = df.shape[0] * df.shape[1]
total_missing = df.isnull().sum().sum()
completion_rate = (total_possible_answers - total_missing) / total_possible_answers * 100
print(f"Overall Survey Completion Rate: {completion_rate:.1f}%")
print(f"Total missing values: {total_missing} out of {total_possible_answers} possible answers")

Overall Survey Completion Rate: 92.8%
Total missing values: 60 out of 832 possible answers


## 3. Response Analysis

### Survey Timeline

In [6]:
# Convert time columns to datetime if they're not already
if 'Startzeit' in df.columns:
    df['Startzeit'] = pd.to_datetime(df['Startzeit'], errors='coerce')
if 'Fertigstellungszeit' in df.columns:
    df['Fertigstellungszeit'] = pd.to_datetime(df['Fertigstellungszeit'], errors='coerce')

# Calculate completion time
if 'Startzeit' in df.columns and 'Fertigstellungszeit' in df.columns:
    df['completion_duration'] = (df['Fertigstellungszeit'] - df['Startzeit']).dt.total_seconds() / 60
    
    print("Survey Completion Statistics:")
    print(f"Average completion time: {df['completion_duration'].mean():.1f} minutes")
    print(f"Median completion time: {df['completion_duration'].median():.1f} minutes")
    print(f"Fastest completion: {df['completion_duration'].min():.1f} minutes")
    print(f"Slowest completion: {df['completion_duration'].max():.1f} minutes")

Survey Completion Statistics:
Average completion time: 58.2 minutes
Median completion time: 29.2 minutes
Fastest completion: 7.5 minutes
Slowest completion: 387.2 minutes


### Professional Background Analysis

In [7]:
# Find role/position column (it has a long German name)
role_col = None
for col in df.columns:
    if 'Rolle in der Praxis' in col:
        role_col = col
        break

if role_col:
    print("Roles mentioned in responses:")
    roles = df[role_col].dropna()
    for i, role in enumerate(roles, 1):
        print(f"Response {i}: {role[:100]}{'...' if len(role) > 100 else ''}")
else:
    print("Role column not found")

Roles mentioned in responses:
Response 1: Zahnaerztin mit diverse adm Aufgaben
Response 2: Oralchirurgin / Zahnärztin; Praxisinhaberin
Response 3: Zahnärztin
Response 4: Dentalsekretärin, Administration und Dentalassistentin, wo es mich gerade braucht
Response 5: Ich arbeite als Praxiskoordinatorin und bin für die Terminvergabe, Kreditoren, Debitoren und Persona...
Response 6: Praxisinhaberin, Zahnärztin
Response 7: Praxisinhaberin, Patientenbehandlung, Planung, Personalmanagment, alle administrativen Aufgaben des ...
Response 8: Praxisadministratorin.
Administrative Arbeiten inkl. dem Tagesablauf  managen
Response 9: Zahnarzt , Einzelpraxis
Response 10: Praxismanagerin
Telefonate, Terminvergabe, Korrespondenz mit Versicherungen etc., Rechnungs- und Mah...
Response 11: Administration, Buchhaltung, Personalwesen
Response 12: Empfang und Labor
Response 13: Praxisinhaber
Response 14: Zahnarzt und Eigentümer der Praxus
Zahnmedizin
Response 15: Zahnarzt in eigener Praxis tätig. 
Behandlung 

### Experience Analysis

In [8]:
# Find experience column
experience_col = None
for col in df.columns:
    if 'Wie lange sind Sie' in col:
        experience_col = col
        break

if experience_col:
    print("Experience levels:")
    experiences = df[experience_col].dropna()
    for i, exp in enumerate(experiences, 1):
        print(f"Response {i}: {exp}")
else:
    print("Experience column not found")

Experience levels:
Response 1: > 5 Jahre
Response 2: > 5 Jahre
Response 3: > 5 Jahre
Response 4: > 5 Jahre
Response 5: > 5 Jahre
Response 6: < 5 Jahre
Response 7: > 5 Jahre
Response 8: > 5 Jahre
Response 9: > 5 Jahre
Response 10: > 5 Jahre
Response 11: > 5 Jahre
Response 12: > 5 Jahre
Response 13: > 5 Jahre
Response 14: > 5 Jahre
Response 15: > 5 Jahre
Response 16: > 5 Jahre
Response 17: > 5 Jahre
Response 18: > 5 Jahre
Response 19: > 5 Jahre
Response 20: > 5 Jahre
Response 21: < 5 Jahre
Response 22: < 5 Jahre
Response 23: > 5 Jahre
Response 24: > 5 Jahre
Response 25: > 5 Jahre
Response 26: > 5 Jahre


### Communication Channels Analysis

In [9]:
# Find communication channels column
comm_col = None
for col in df.columns:
    if 'Über welche Kanäle kontaktieren' in col:
        comm_col = col
        break

if comm_col:
    print("Communication channels used:")
    channels = df[comm_col].dropna()
    for i, channel in enumerate(channels, 1):
        print(f"Response {i}: {channel}")
else:
    print("Communication channels column not found")

Communication channels used:
Response 1: Online Formular;Telefon;Persönlich vor Ort;Email;
Response 2: Email;Telefon;
Response 3: Telefon;Persönlich vor Ort;
Response 4: Telefon;Email;Online Formular;
Response 5: Telefon;Email;Online Formular;
Response 6: Telefon;Email;Persönlich vor Ort;
Response 7: Email;Persönlich vor Ort;Telefon;
Response 8: Telefon;Email;Persönlich vor Ort;Messanger Dienste wie What's App, Telegram etc.;
Response 9: Telefon;Email;
Response 10: Telefon;Email;Persönlich vor Ort;
Response 11: Telefon;Email;Persönlich vor Ort;SMS zur Erinnerung;
Response 12: Telefon;
Response 13: Telefon;Email;Persönlich vor Ort;
Response 14: Telefon;Persönlich vor Ort;
Response 15: Telefon;Email;Persönlich vor Ort;
Response 16: Telefon;Email;
Response 17: Telefon;Email;Persönlich vor Ort;
Response 18: Email;Telefon;Persönlich vor Ort;
Response 19: Telefon;Email;
Response 20: Telefon;Briefpost;
Response 21: Telefon;Email;
Response 22: Telefon;Email;
Response 23: Telefon;Email;
Respons

### Software Usage Analysis

In [10]:
# Find software column
software_col = None
for col in df.columns:
    if 'Praxisverwaltungssoftware' in col:
        software_col = col
        break

if software_col:
    print("Practice Management Software used:")
    software = df[software_col].dropna()
    for i, sw in enumerate(software, 1):
        print(f"Response {i}: {sw}")
        
    # Simple analysis of mentioned software
    software_mentions = {}
    for sw in software:
        sw_lower = sw.lower()
        if 'zawin' in sw_lower:
            software_mentions['ZaWin'] = software_mentions.get('ZaWin', 0) + 1
        elif 'charly' in sw_lower:
            software_mentions['Charly'] = software_mentions.get('Charly', 0) + 1
        elif 'evident' in sw_lower:
            software_mentions['Evident'] = software_mentions.get('Evident', 0) + 1
    
    if software_mentions:
        print("\nSoftware usage summary:")
        for sw, count in software_mentions.items():
            print(f"{sw}: {count} mention(s)")
else:
    print("Software column not found")

Practice Management Software used:
Response 1: ZaWin
Response 2: ZaWin
Response 3: ZaWIn
Response 4: Aeskulap Kern Concept
Response 5: ErgoDent
Response 6: ZaWin
Response 7: ZaWin
Response 8: Ergodent, Sidexis
Response 9: Zawin
Response 10: ZaWin
Response 11: Ergodent von CCS
Response 12: Dentalmed XP
Response 13: ZaWin
Response 14: Dental med XP
Response 15: Dent II 
Response 16: aeskulap
Response 17: ProzessTeam
Response 18: ZaWin
Response 19: -
Response 20: Keine
Response 21: Zawin
Response 22: ZaWin
Response 23: Kern Concept - Aeskulap
Response 24: Dental med XP
Response 25: ZaWin
Response 26: ZaWin

Software usage summary:
ZaWin: 13 mention(s)


## 4. Key Challenges and Pain Points

In [11]:
# Find the main challenges/pain points columns
challenge_cols = []
for col in df.columns:
    if any(keyword in col.lower() for keyword in ['störfaktoren', 'herausforderungen', 'mühsam', 'zeitaufwendig']):
        challenge_cols.append(col)

print(f"Found {len(challenge_cols)} columns related to challenges:")
for i, col in enumerate(challenge_cols, 1):
    print(f"{i}. {col[:80]}{'...' if len(col) > 80 else ''}")
    
    # Show responses for this challenge
    responses = df[col].dropna()
    if len(responses) > 0:
        print("   Responses:")
        for j, response in enumerate(responses, 1):
            print(f"   Response {j}: {response[:150]}{'...' if len(response) > 150 else ''}")
    print()

Found 3 columns related to challenges:
1. Was sind die grössten "Störfaktoren" oder Unterbrechungen bei Ihrer täglichen ad...
   Responses:
   Response 1: Die Aufgaben und Fragen die nur ich antworten kann
   Response 2: Qualitätsmangement, Versicherungsanfragen
   Response 3: das Personal
   Response 4: Telefon, neuer Termin vergeben
   Response 5: Personal, wo Fragen hat, Vorgesetzte die sonst noch Wünsche oder Probleme haben. Patienten die ganze Planung ändern möchten. 
   Response 6: Nächster Patient, Rückfragen von Mitarbeitern
   Response 7: Die administrative Arbeit wird zwischen den Patientenbehandlungen oder Abends oder am Wochenende erledigt, es ist kein "Bürotag" bestimmt. Dadurch gib...
   Response 8: MA kommen ständig was fragen, Telefone, Patientenanfragen vor Ort, Krankmeldungen, neue Planungen etc..
   Response 9: Telefon, Patientennotfall
   Response 10: Telefonate und Terminvergabe gleichzeitig, oder man hat einen recht anspruchsvollen Antrag an die Versicherung zu sc

## 5. Digital Transformation Insights

In [12]:
# Find digital transformation related columns
digital_cols = []
for col in df.columns:
    if any(keyword in col.lower() for keyword in ['digital', 'online', 'automatisch', 'ki', 'assistent']):
        digital_cols.append(col)

print(f"Found {len(digital_cols)} columns related to digital transformation:")
for i, col in enumerate(digital_cols, 1):
    print(f"{i}. {col[:80]}{'...' if len(col) > 80 else ''}")
    
    # Show responses
    responses = df[col].dropna()
    if len(responses) > 0:
        print("   Responses:")
        for j, response in enumerate(responses, 1):
            print(f"   Response {j}: {response[:150]}{'...' if len(response) > 150 else ''}")
    print()

Found 4 columns related to digital transformation:
1. Welche anderen digitalen Tools oder Programme nutzen Sie regelmässig im administ...
   Responses:
   Response 1: BESR
   Response 2: Trios, Rö-Software Orange
   Response 3: alle die wir benötigen 
   Response 4: Mediport, Byzz
   Response 5: Outlook, Swisstransfer, Condent
   Response 6: Digitale Anamnese am Tablet, Scanner der Sterilisationsetiketten, Nostic, Onlineagenda, Materialbestellung Novadent
   Response 7: Excel, Word, Acrobat, OnyxCeph, Sidexis, Outlook
   Response 8: Anyguard, Diagnokat, Nostic, Q.Wiki
   Response 9: Digit. Röntgen, Cerec
   Response 10: Romexis (Röntgenprogramm)
Onyx Ceph (Auswertungsprogramm von Röntgen und Modellen)
Outlook
   Response 11: Abaweb, MedMonitor, Sidexis, DS Core
   Response 12: dig. RX-Programm von Curaden
   Response 13: Sidexis /Outlook/ Office-Produkte
   Response 14: Excel, Word
   Response 15: Windows, Carestream CS, Mailprogramme, Abrechnungsprogramme (Mediport)
   Response 16: Di